In [2]:
#!/usr/bin/env python3
import numpy as np
import warnings
from numpy.typing import NDArray

from typing import Optional, Tuple, Union
from osgeo import gdal, osr, gdal_array
from pathlib import Path


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import pandas as pd
import h5py
import fsspec
from pyproj import Proj, CRS
import matplotlib.pyplot as plt
import shapely.wkt as wkt
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
from rasterio import merge
import folium
from folium import plugins
import rioxarray
import sys
sys.path.append('../../')
from src.cslc_utils import get_s3path, read_cslc, cslc_info, rasterWrite, custom_merge, colorize, getbasemaps, moving_window_mean

import warnings
warnings.filterwarnings('ignore')

In [5]:
data_dir = 's3://opera-pst-rs-pop1/products/CSLC_S1'
burst_id = ['T144-308014-IW2', 
            'T144-308015-IW2']

# get list of dates
b1paths = (open(f"{burst_id[0]}.txt","r").read().split("\n"))
b2paths = (open(f"{burst_id[1]}.txt","r").read().split("\n"))
dates = [path[40:48] for path in b1paths]
nd = len(dates)

# send to dataframe
file_df = pd.DataFrame({'Date':dates,f'{burst_id[0]}':b1paths,f'{burst_id[1]}':b2paths})

In [6]:
# Read before and after event dataset 
before = []; after = []  

for id in burst_id: 
    path_h5 = file_df[f'{id}'][0][:-1]
    dat = read_cslc(f'{data_dir}/{path_h5}/{path_h5}.h5')
    before.append(dat)

for id in burst_id:  
    path_h5 = file_df[f'{id}'][1][:-1]
    dat = read_cslc(f'{data_dir}/{path_h5}/{path_h5}.h5')
    after.append(dat)

Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151127T135951Z_v0.1_20230708T230140Z/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151127T135951Z_v0.1_20230708T230140Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151127T135954Z_v0.1_20230708T230140Z/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151127T135954Z_v0.1_20230708T230140Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151221T135950Z_v0.1_20230708T230145Z/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151221T135950Z_v0.1_20230708T230145Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151221T135953Z_v0.1_20230708T230145Z/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151221T135953Z_v0.1_20230708T230145Z.h5


In [30]:
path_h5 = file_df[f'{burst_id[0]}'][0][:-1]
xcoor, ycoor, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(f'{data_dir}/{path_h5}/{path_h5}.h5')
cslc_poly = wkt.loads(bounding_polygon)
bbox = [cslc_poly.bounds[0], cslc_poly.bounds[2], cslc_poly.bounds[1], cslc_poly.bounds[3]]
bbox1 = bbox

In [31]:
path_h5 = file_df[f'{burst_id[1]}'][0][:-1]
xcoor, ycoor, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(f'{data_dir}/{path_h5}/{path_h5}.h5')
cslc_poly = wkt.loads(bounding_polygon)
bbox = [cslc_poly.bounds[0], cslc_poly.bounds[2], cslc_poly.bounds[1], cslc_poly.bounds[3]]
bbox2 = bbox

In [25]:
from pyproj import Transformer

transformer = Transformer.from_crs(f'EPSG:{epsg}', 'EPSG:4326')
lat0, lon0 = transformer.transform(0, 0)
dlat, dlon = transformer.transform(dx, dy)
dlat = np.abs(dlat - lat0)
dlon = np.abs(dlon - lon0)
latlon_spacing = [dlat,dlon]

In [23]:
dlon

4.4794979231710386e-05

In [24]:
dlat

9.019376366698369e-05

In [26]:
def lalo2xy(lat: np.float32,
            lon: np.float32,
            data_snwe: list,
            latlon_step: list,
            rounding_method: Optional[str] = 'floor') \
        -> Tuple[np.int16, np.int16]:
    """
    Georeferenced coordinates to image space coordinates.
    GDAL raster starting point is the upper left corner.

    Parameters
    ----------
    lat : float
        search latitude
    lon : float
        search longitude
    data_snwe : list
        [South, North, West, East] bounds of raster
        North (y0) and West(x0) as reference point
    latlon_step : list
        pixel spacing [latitude_spacing, longitude_spacing]
    rounding_method : str
        rounding method, default is 'floor' other option is 'around'
        Read notes below. TODO. test different rounding routines

    Returns
    -------
    x : int
        coordinate in image space (x-axis/columns, direction of width)
        from x0 (top up column)
    y : int
        coordinate in image space (y-axis/rows, direction of length)
        from y0 (top left row)
    """

    # np.floor works better with points and raster - Need to check why
    # but with two rasters sometimes one pixel is missing or is redundant
    if rounding_method == 'floor':
        x = int(np.floor((lon - data_snwe[2]) / latlon_step[1] + 0.01))
        y = int(np.floor((lat - data_snwe[1]) / latlon_step[0] + 0.01))

    # np.around works better with two rasters
    # test it out, I think it has something to how numpy floor is
    # rounding negative values
    # example np.around(-125.2) = -125 np.floor(-125.2) = -126
    # np.around(125.6) = 126, np.floor(125.6) = 125
    elif rounding_method == 'around':
        x = int(np.around((lon - data_snwe[2]) / latlon_step[1] + 0.01))
        y = int(np.around((lat - data_snwe[1]) / latlon_step[0] + 0.01))

    return x, y


In [33]:
frame_overlap(bbox1,bbox2,latlon_spacing,latlon_spacing)

((slice(-417, 820, None), slice(0, 177, None)),
 (slice(0, 1237, None), slice(3702, 3879, None)))

In [32]:
# Extract overlap bounds
def frame_overlap(snwe1: list,
                  snwe2: list,
                  latlon_step1: list,
                  latlon_step2: list,
                  latlon_step: Optional[list] = [-0.000833334, 0.000833334]
                  ) -> Tuple[tuple, tuple]:
    """
    Parameters
    ----------
    Use raster metadata to find overlap between two Images
    snwe1 : list
        [South, North, West, East] bounds of Image-1
    snwe2 : list
        [South, North, West, East] bounds of Image-2
    latlon_step1 : list
        latitude and longitude pixel spacing of Image-1
    latlon_step2 : list
        latitude and longitude pixel spacing of Image-2

    Returns
    -------
    subset1 : np.slice
        Intersection subset [y1:y2, x1:x2] for the Image-1
    subset2 : np.slice
        Intersection subset [y1:y2, x1:x2] for the Image-2
    """

    snwe = np.vstack([snwe1, snwe2])
    # Find overlap bounds
    overlap_snwe = [np.max(snwe[:, 0]), np.min(snwe[:, 1]),
                    np.max(snwe[:, 2]), np.min(snwe[:, 3])]

    # Georeferenced space to image coordinate space
    # Frame-1
    x1, y1 = lalo2xy(overlap_snwe[1], overlap_snwe[2], snwe1, latlon_step1)
    # Frame-2
    x2, y2 = lalo2xy(overlap_snwe[1], overlap_snwe[2], snwe2, latlon_step2)

    # Overlap bounds - force overlaps to have same dimensions
    # latlon_spacing sometimes diff at 13th decimal
    length = int(round((overlap_snwe[0] - overlap_snwe[1]) / latlon_step[0]))
    width = int(round((overlap_snwe[3] - overlap_snwe[2]) / latlon_step[1]))

    subset1 = np.s_[y1:y1 + length, x1:x1 + width]
    subset2 = np.s_[y2:y2 + length, x2:x2 + width]

    return subset1, subset2


In [37]:
[before[0],before[1]]

[array([[nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        ...,
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj]],
       dtype=complex64),
 array([[nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        ...,
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj],
        [nan+nanj, nan+nanj, nan+nanj, ..., nan+nanj, nan+nanj, nan+nanj]],
       dtype=complex64)]

In [35]:
combine_data_to_single([before[0],before[1]],[bbox1,bbox2],[latlon_spacing,latlon_spacing])

ValueError: could not broadcast input array from shape (4881,20652) into shape (1325,0)

In [38]:
np.shape(before[0])

(4881, 20652)

In [34]:
def combine_data_to_single(data_list: list,
                           snwe_list: list,
                           latlon_step_list: list,
                           method: Optional[str] = 'mean',
                           latlon_step: Optional[list] =
                           [-0.000833334, 0.000833334]) \
        -> Tuple[NDArray, NDArray, list]:
    """
    Merge multiple arrays to one array. Combine them in ndarray, then apply
    function along the n_layers axis

    Parameters
    ----------
    data_list : list
        list of arrays containing raster values
    snwe_list : list
        list of arrays containing snwe (extent) values
    latlon_step_list : list
        list of arrays containing pixel spacing in lat and lon direction
        for each dataset
    method : str
        method to merge overlapping pixes, use mean, min, max etc..
        TODO: need to refine this part of code

    Returns
    -------
    comb_data : ndarray
        combined data [n_frames, length, width]
    SNWE : array
        extent of the combined data
    latlon_step : array
        pixel spacing in lat, lon of combined data

    """
    # Get the maximum extent of all data
    n = len(data_list)
    snwe_all = np.squeeze([snwe for snwe in snwe_list])

    SNWE = np.array([np.min(snwe_all[:, 0]), np.max(snwe_all[:, 1]),
                     np.min(snwe_all[:, 2]), np.max(snwe_all[:, 3])]).T

    length = abs(int(np.around((SNWE[1] - SNWE[0]) / latlon_step[0] + 0.01)))
    width = abs(int(np.around((SNWE[2] - SNWE[3]) / latlon_step[1] + 0.01)))

    # create combined data array
    # handle if 3D metadata layer
    if len(data_list[0].shape) > 2:
        comb_data = np.empty((n, data_list[0].shape[0], length, width),
                             dtype=np.float64) * np.nan
    else:
        comb_data = np.empty((n, length, width), dtype=np.float64) * np.nan
    for i, data in enumerate(data_list):
        x, y = np.abs(lalo2xy(SNWE[1], SNWE[2], snwe_list[i],
                              latlon_step_list[i], 'around'))
        # handle if 3D metadata layer
        if len(data.shape) > 2:
            comb_data[i, 0:data.shape[0], y:y+data.shape[1],
                      x: x+data.shape[2]] = data
        else:
            comb_data[i, y:y+data.shape[0], x: x+data.shape[1]] = data

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        # combine using numpy
        if method == 'mean':
            comb_data = np.nanmean(comb_data, axis=0)
        elif method == 'median':
            comb_data = np.nanmedian(comb_data, axis=0)
        elif method == 'min':
            comb_data = np.nanmin(comb_data, axis=0)
        elif method == 'max':
            comb_data = np.nanmax(comb_data, axis=0)

    return comb_data, SNWE,  latlon_step
